# Credit Card Retention Analysis

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot
sns.set()
pd.options.display.max_columns = 999

In [80]:
data = pd.read_csv('../data/BankChurners_v2.csv')

In [10]:
data = data[['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',]]

In [14]:
data['Education_Level'] = data['Education_Level'].fillna('Unknown')
data['Marital_Status'] = data['Marital_Status'].fillna('Unknown')
data['Income_Category'] = data['Income_Category'].fillna('Unknown')

In [18]:
# https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-5-binning-c5bd5fd1b950
bins = [25, 30, 40, 50, 60, 70, 80]
labels = ['20s', '30s', '40s', '50s', '60s', '70s']
data['Customer_Age_bins'] = pd.cut(data['Customer_Age'], bins=bins, labels=labels, include_lowest=True, right=False)

***

## Data Visualization: Data Tables

**Aggregations and Groupby**: If you are familiar with `Pivot Tables` in excel, than the way that `.groupby()` works with pandas dataframes should be pretty intuative! What both of these functions will try to accomplish is aggregations and ways of summarizing the data. They are both flexible enough to allow you to pull in different aggregation types (sum, mean, counts, etc). To read more about the difference between `pivot tables` in python and `groupby`, see this [guide](https://levelup.gitconnected.com/pivot-tables-in-pandas-7b672e6d8f47).

Coming back to our business problem, we want to understand something about Customers who are leaving. Let's see if we can see anything obvious about their behavior:

In [58]:
data.groupby([ 'Attrition_Flag']).agg({
               'CLIENTNUM':'nunique',
               'Customer_Age': 'mean', 
               'Dependent_count': 'mean',
               'Months_on_book': 'mean',
               'Total_Relationship_Count': 'mean',
               'Months_Inactive_12_mon': 'mean',
               'Contacts_Count_12_mon': 'mean',
               'Credit_Limit': 'mean',
               'Total_Revolving_Bal': 'mean',
               'Avg_Open_To_Buy': 'mean',
               'Total_Amt_Chng_Q4_Q1': 'mean',
               'Total_Trans_Amt': 'mean',
               'Total_Trans_Ct': 'mean',
               'Total_Ct_Chng_Q4_Q1': 'mean',
               'Avg_Utilization_Ratio': 'mean'
            })

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
Attrition_Flag,,,,,,,,,,,,,,,
Attrited Customer,1627,46.659496,2.402581,36.178242,3.279656,2.693301,2.972342,8136.039459,672.822987,7463.216472,0.694277,3095.025814,44.933620,0.554386,0.162475
Existing Customer,8500,46.262118,2.335412,35.880588,3.914588,2.273765,2.356353,8726.877518,1256.604118,7470.273400,0.772510,4654.655882,68.672588,0.742434,0.296412


To see the groups more clearly, we can use the `Transpose` function. This will swap Columns and Rows.

Styling a dataframe [guide](https://towardsdatascience.com/style-pandas-dataframe-like-a-master-6b02bf6468b0). Row-wise, [here](https://stackoverflow.com/questions/52783419/format-pandas-dataframe-row-wise)

In [59]:
data_pivot = data.groupby(['Attrition_Flag']).agg({
                           'CLIENTNUM':'nunique', # number of unique customers in each group
                           'Customer_Age': 'mean', # the rest are looking at the mean per group
                           'Dependent_count': 'mean',
                           'Months_on_book': 'mean',
                           'Total_Relationship_Count': 'mean',
                           'Months_Inactive_12_mon': 'mean',
                           'Contacts_Count_12_mon': 'mean',
                           'Credit_Limit': 'mean',
                           'Total_Revolving_Bal': 'mean',
                           'Avg_Open_To_Buy': 'mean',
                           'Total_Amt_Chng_Q4_Q1': 'mean',
                           'Total_Trans_Amt': 'mean',
                           'Total_Trans_Ct': 'mean',
                           'Total_Ct_Chng_Q4_Q1': 'mean',
                           'Avg_Utilization_Ratio': 'mean'
                        })

data_pivot

Attrition_Flag,Attrited Customer,Existing Customer
CLIENTNUM,1627.000000,8500.000000
Customer_Age,46.659496,46.262118
Dependent_count,2.402581,2.335412
Months_on_book,36.178242,35.880588
Total_Relationship_Count,3.279656,3.914588
Months_Inactive_12_mon,2.693301,2.273765
Contacts_Count_12_mon,2.972342,2.356353
Credit_Limit,8136.039459,8726.877518
Total_Revolving_Bal,672.822987,1256.604118
Avg_Open_To_Buy,7463.216472,7470.273400


Instead of trying to understand the differences by visual inspection, let's calculate the differences between groups and sort to see where they differ the most.

In [60]:
data_pivot['Diff'] = data_pivot['Attrited Customer'] / data_pivot['Existing Customer'] - 1

data_pivot.sort_values('Diff')

Attrition_Flag,Attrited Customer,Existing Customer,Diff
CLIENTNUM,1627.000000,8500.000000,-0.808588
Total_Revolving_Bal,672.822987,1256.604118,-0.464570
Avg_Utilization_Ratio,0.162475,0.296412,-0.451860
Total_Trans_Ct,44.933620,68.672588,-0.345683
Total_Trans_Amt,3095.025814,4654.655882,-0.335069
Total_Ct_Chng_Q4_Q1,0.554386,0.742434,-0.253286
Total_Relationship_Count,3.279656,3.914588,-0.162196
Total_Amt_Chng_Q4_Q1,0.694277,0.772510,-0.101271
Credit_Limit,8136.039459,8726.877518,-0.067703
Avg_Open_To_Buy,7463.216472,7470.273400,-0.000945


We can see that the `Total_Revolving_Bal` and `Avg_Utilization_Ratio` along with `Total_Trans_Ct` and `Total_Trans_Amt` show the biggest differences, all showing 30% or more below Existing Customers. 